# Cloud sandbox powered by FIWARE

## About the open source GPL3 license and copyright for this product

Copyright (c) 2024 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this course until you have built your own platform with these tools, 
separate from the computate.org platform, and reconfigure your fork of this repo to deploy 
your own platform instead of the computate.org platform. 

QUESTIONS

For questions about this open source license, please contact our public mailing list at computate@group.computate.org


## Switch to the Python 3.9 Kernel

When opening this Jupyter Notebook in VSCode, you will need to switch to the Jupyter 3.9 Kernel. 

## Deploy the Scorpio Broker in the OpenShift Developer Sandbox

A FIWARE Context Broker like ScorpioBroker is required for storing smart device entity data of vehicle traffic cameras and pedestrian traffic cameras in the project. 

## Create a Scorpio Broker secret to connect to PostgreSQL

In [ ]:
%%bash
oc create secret generic postgres-pguser-scorpiobroker \
  --from-literal password="$(tr -dc 'A-Za-z0-9' < /dev/urandom | head -c 16)"
echo DONE

## Create database user for scorpiobroker on PostgreSQL

In [ ]:
%%bash
oc exec pod/postgresql-0 -- env PGPASSWORD=$(oc get secret/postgresql -o jsonpath={.data.postgres-password} | base64 -d) psql -U postgres postgres -c \
  "create user scorpiobroker password '$(oc get secret/postgres-pguser-scorpiobroker -o jsonpath={.data.password} | base64 -d)' superuser"
echo DONE

## Create database for scorpiobroker on PostgreSQL

In [ ]:
%%bash
oc exec pod/postgresql-0 -- env PGPASSWORD=$(oc get secret/postgresql -o jsonpath={.data.postgres-password} | base64 -d) psql -U postgres postgres -c \
  "create database scorpiobroker owner scorpiobroker"
echo DONE

## Deploy the Scorpio Broker Deployment

In [ ]:
%%bash
ansible-playbook playbooks/deploy-scorpiobroker.yaml \
  -e "DBHOST=postgresql" \
  -e "DBPORT=5432" \
  -e "DBNAME=scorpiobroker" \
  -e "DBUSER=scorpiobroker" \
  -e "DB_SECRET_NAME=postgres-pguser-scorpiobroker" \
  -e "DB_SECRET_KEY=password" \
  -e "SCORPIOBROKER_PORT=9090" \
  -e "SCORPIOBROKER_REPLICAS=1" \
  -e "SCORPIOBROKER_LIMITS_CPU=1" \
  -e "SCORPIOBROKER_LIMITS_MEMORY=3Gi" \
  -e "SCORPIOBROKER_REQUESTS_CPU=1" \
  -e "SCORPIOBROKER_REQUESTS_MEMORY=3Gi" \
  -e "SCORPIOBROKER_NAMESPACE=$(oc project -q)"
echo DONE

### View Scorpio Broker pod details
After deploying Scorpio Broker with helm, it will take a minute before the Scorpio Broker pod is up and running. Run the command below until the Scorpio Broker pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc get pod -l app.kubernetes.io/instance=scorpiobroker
oc wait pod -l app.kubernetes.io/instance=scorpiobroker --for=condition=Ready --timeout=5m
oc get pod -l app.kubernetes.io/instance=scorpiobroker
echo DONE

### View Scorpio Broker pod logs
If your Scorpio Broker pod does not reach the STATUS Running, you can run the command below to view the pod logs of Scorpio Broker and check for other errors that may have occured. 

In [ ]:
%%bash
oc logs -l app.kubernetes.io/name=scorpiobroker
echo DONE

## Next...
If you have successfully ran all of the commands above, congratulations, you are ready to move on to the next notebook in the course. 

- If you have additional questions or issues, please [create an issue for the course here](https://github.com/computate-org/computate/issues). 
- Otherwise, please continue to the next document [09-deploy-iotagent-json.ipynb](09-deploy-iotagent-json.ipynb). 
